In [2]:
import numpy as np
import pyodbc
import pandas as pd

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10048\1098552811.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# current
# connect to the SQL server database
conn = pyodbc.connect(
    '''
    DRIVER={ODBC Driver 17 for SQL Server};
    SERVER=DESKTOP-7CB1RAA;
    DATABASE=CompanyX;
    Trusted_Connection=yes;
    '''
    )

# create a cursor to fetch data
cursor = conn.cursor()

### Extract the table 1
Each row contain <Tid, s, t>, Tid is the transaction identifier, s is the set of item, t is the time epoch expressing the time that Tid occurs

![Table 1](img/table1.png)


In [4]:
query1 = """
select SalesOrderID as TiD,
	ProductID as Item
from [CompanyX].[Sales].[SalesOrderDetail]
group by SalesOrderID, ProductID
order by SalesOrderID
"""

cursor.execute(query1)
result = cursor.fetchall()
df1 = pd.DataFrame(np.asarray(result), columns=['TiD','Item'])
print(df1)


          TiD  Item
0       43659   716
1       43659   712
2       43659   771
3       43659   778
4       43659   774
...       ...   ...
121312  75122   712
121313  75122   878
121314  75123   712
121315  75123   879
121316  75123   878

[121317 rows x 2 columns]


In [5]:
# merge all the item with same TiD into one row
df1 = df1.groupby('TiD')['Item'].apply(list).reset_index()
print(df1)

         TiD                                               Item
0      43659  [716, 712, 771, 778, 774, 709, 777, 711, 773, ...
1      43660                                         [762, 758]
2      43661  [743, 712, 708, 716, 775, 747, 711, 742, 778, ...
3      43662  [730, 722, 726, 763, 753, 725, 738, 749, 733, ...
4      43663                                              [760]
...      ...                                                ...
31460  75119                                    [873, 921, 930]
31461  75120                                    [884, 878, 712]
31462  75121                                    [921, 707, 930]
31463  75122                                         [712, 878]
31464  75123                                    [712, 879, 878]

[31465 rows x 2 columns]


In [6]:
query2 = """
select SalesOrderID as TiD,
	OrderDate as EpocTime
from [CompanyX].[Sales].[SalesOrderHeader]
group by SalesOrderID, OrderDate 
order by SalesOrderID
"""

cursor.execute(query2)
result = cursor.fetchall()
df2 = pd.DataFrame(np.asarray(result), columns=['TiD','EpocTime'])
print(df2)

         TiD   EpocTime
0      43659 2011-05-31
1      43660 2011-05-31
2      43661 2011-05-31
3      43662 2011-05-31
4      43663 2011-05-31
...      ...        ...
31460  75119 2014-06-30
31461  75120 2014-06-30
31462  75121 2014-06-30
31463  75122 2014-06-30
31464  75123 2014-06-30

[31465 rows x 2 columns]


In [7]:
# perform inner join on the two dataframes
table1 = pd.merge(df1, df2, on='TiD', how='inner')
print(table1)   

         TiD                                               Item   EpocTime
0      43659  [716, 712, 771, 778, 774, 709, 777, 711, 773, ... 2011-05-31
1      43660                                         [762, 758] 2011-05-31
2      43661  [743, 712, 708, 716, 775, 747, 711, 742, 778, ... 2011-05-31
3      43662  [730, 722, 726, 763, 753, 725, 738, 749, 733, ... 2011-05-31
4      43663                                              [760] 2011-05-31
...      ...                                                ...        ...
31460  75119                                    [873, 921, 930] 2014-06-30
31461  75120                                    [884, 878, 712] 2014-06-30
31462  75121                                    [921, 707, 930] 2014-06-30
31463  75122                                         [712, 878] 2014-06-30
31464  75123                                    [712, 879, 878] 2014-06-30

[31465 rows x 3 columns]


In [8]:
# export the table to a csv file
# table1.to_csv('table1.csv', index=False)
# TODO: one hot encoding for the items later

### Extract the table 2
Each row contain <i, dr, t>, i is the item, dr is the discount rate, t is the period in which item i has dr discount

![Table 2](img/table2.png)

In [9]:
querry3 = """
select distinct
	sod.ProductID as item,
	so.DiscountPct as dr,
	so.StartDate as startDate,
	so.EndDate as endDate
from [CompanyX].[Sales].[SalesOrderDetail] as sod
join [CompanyX].[Sales].[SpecialOffer] as so
on sod.SpecialOfferID = so.SpecialOfferID
order by sod.ProductID
"""

cursor.execute(querry3)
result = cursor.fetchall()
df3 = pd.DataFrame(np.asarray(result), columns=['item','dr','startDate','endDate'])
df3['dr'] = df3['dr'].astype(float)

table2 = df3
print(table2)
# type(df3['startDate'][0])

    item    dr  startDate    endDate
0    707  0.00 2011-05-01 2014-11-30
1    707  0.02 2011-05-31 2014-05-30
2    707  0.05 2011-05-31 2014-05-30
3    707  0.10 2012-05-30 2012-06-29
4    707  0.15 2013-05-30 2013-06-29
..   ...   ...        ...        ...
479  998  0.00 2011-05-01 2014-11-30
480  998  0.02 2011-05-31 2014-05-30
481  998  0.05 2011-05-31 2014-05-30
482  999  0.00 2011-05-01 2014-11-30
483  999  0.02 2011-05-31 2014-05-30

[484 rows x 4 columns]


In [10]:
# export the table to a csv file
# table2.to_csv('table2.csv', index=False)

In [21]:
query4 = """
SELECT 
        [ProductID]
      ,[StartDate]
      ,[EndDate]
      ,[ListPrice]
   FROM [CompanyX].[Production].[ProductListPriceHistory]
"""
cursor.execute(query4)
result = cursor.fetchall()
df4 = pd.DataFrame(np.asarray(result), columns=['ProductID','StartDate','EndDate','Price'])
print(df4)
# df4.to_csv('price.csv', index=False)

    ProductID  StartDate    EndDate     Price
0         707 2011-05-31 2012-05-29   33.6442
1         707 2012-05-30 2013-05-29   33.6442
2         707 2013-05-30        NaT   34.9900
3         708 2011-05-31 2012-05-29   33.6442
4         708 2012-05-30 2013-05-29   33.6442
..        ...        ...        ...       ...
390       995 2013-05-30        NaT  101.2400
391       996 2013-05-30        NaT  121.4900
392       997 2013-05-30        NaT  539.9900
393       998 2013-05-30        NaT  539.9900
394       999 2013-05-30        NaT  539.9900

[395 rows x 4 columns]


In [24]:
query5 = """
SELECT [ProductID]
      ,[StartDate]
      ,[EndDate]
      ,[StandardCost]
  FROM [CompanyX].[Production].[ProductCostHistory]
"""
cursor.execute(query5)
result = cursor.fetchall()
df5 = pd.DataFrame(np.asarray(result), columns=['ProductID','StartDate','EndDate','Cost'])
print(df5)
# df5.to_csv('cost.csv', index=False)

    ProductID  StartDate    EndDate      Cost
0         707 2011-05-31 2012-05-29   12.0278
1         707 2012-05-30 2013-05-29   13.8782
2         707 2013-05-30        NaT   13.0863
3         708 2011-05-31 2012-05-29   12.0278
4         708 2012-05-30 2013-05-29   13.8782
..        ...        ...        ...       ...
390       995 2013-05-30        NaT   44.9506
391       996 2013-05-30        NaT   53.9416
392       997 2013-05-30        NaT  343.6496
393       998 2013-05-30        NaT  343.6496
394       999 2013-05-30        NaT  343.6496

[395 rows x 4 columns]
